In [1]:
import os

from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip uninstall -y bitsandbytes
!pip install --no-cache-dir bitsandbytes
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 289.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 131.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 144.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 141.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 135.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 160.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 179.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 220.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82

In [4]:
pip install np faiss-cpu sentence-transformers peft rouge_score bert_score pymupdf4llm pdfplumber rank_bm25 evaluate

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

In [5]:
#!/usr/bin/env python
import argparse
import json
import time
import re
import os
import csv
import numpy as np
import torch
import faiss
import matplotlib.pyplot as plt
import pandas as pd
import string
from pathlib import Path
from typing import List, Dict
from collections import defaultdict
import evaluate


from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from peft import LoraConfig, get_peft_model
from tqdm import tqdm



# Use your PDF-to-markdown converter; here we use pymupdf4llm as in your snippet.
import pymupdf4llm

# For alternative PDF extraction (if needed)
import pdfplumber



# For TF-IDF retrieval:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# For BM25 retrieval:
from rank_bm25 import BM25Okapi

# ========================================================
# 1. Mount Google Drive
# ========================================================

from google.colab import drive
drive.mount('/content/drive')



##############################
# CONFIGURATION
##############################

# Maximum token limit for each chunk (set below 512 to allow room for question tokens and special tokens)
MAX_CHUNK_TOKENS = 400

# Choose a tokenizer for counting tokens (using a BERT tokenizer as an example)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")




Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:

# In Google Colab, if your notebook is in /content and the files are as above:
BASE_DIR = Path("/content/drive/My Drive/Dissertation/phase_2/generative")
AGREEMENTS_DIR = BASE_DIR / "agreements"
GOLD_STANDARD_JSON = BASE_DIR / "gold_standard.json"
PROCESSED_DIR = BASE_DIR / "processed"         # output for processed main PDFs
IMAGES_DIR = BASE_DIR / "images"



print("Base Directory:", BASE_DIR)
print("Agreements Directory:", AGREEMENTS_DIR)
print("Gold Standard JSON:", GOLD_STANDARD_JSON)


Base Directory: /content/drive/My Drive/Dissertation/phase_2/generative
Agreements Directory: /content/drive/My Drive/Dissertation/phase_2/generative/agreements
Gold Standard JSON: /content/drive/My Drive/Dissertation/phase_2/generative/gold_standard.json


# PDF PREPROCESSING & MARKDOWN EXTRACTION

In [8]:
def pdf_to_markdown(pdf_path: Path) -> str:

    """
    Convert a PDF to markdown using pymupdf4llm.
    If there are issues with pymupdf4llm, consider switching to pdfplumber.
    """

    return pymupdf4llm.to_markdown(str(pdf_path))

def clean_text(text: str) -> str:

    """
    Remove unwanted artifacts and non-ASCII characters.
    """

    text = re.sub(r'â', '', text)
    text = re.sub(r'\*+', '', text)
    text = re.sub(r'â*', '', text)
    text = text.encode("ascii", errors="ignore").decode("ascii")

    return text.strip()

def split_markdown_by_headers(markdown: str) -> List[Dict[str, str]]:

    """
    Splits markdown text into sections based on headers.
    Returns a list of dicts with keys 'title' and 'content'.
    """

    sections = []
    current_section = {"title": None, "content": ""}

    for line in markdown.splitlines():

        header_match = re.match(r'^(#{1,6})\s+(.*)', line)
        if header_match:
            if current_section["title"] is not None or current_section["content"].strip():
                sections.append(current_section)
            title = header_match.group(2).strip()
            current_section = {"title": title, "content": ""}
        else:
            current_section["content"] += line + "\n"

    if current_section["title"] is not None or current_section["content"].strip():
        sections.append(current_section)

    return sections

def process_content(text: str) -> str:

    """
    Replace newline characters with a space and collapse extra spaces.
    """

    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def preprocess_markdown_file(file_path: Path) -> List[Dict[str, str]]:

    """
    Process a markdown file: clean, split by headers, process content,
    and filter out trivial sections.
    """

    with open(file_path, 'r', encoding='utf-8') as f:
        markdown = f.read()
    markdown = clean_text(markdown)
    sections = split_markdown_by_headers(markdown)

    for sec in sections:
        sec["content"] = process_content(sec["content"])

    # Filter out sections that are too trivial
    filtered_sections = []

    for sec in sections:

        if not sec["content"].strip():
            continue
        filtered_sections.append(sec)

    return filtered_sections

def save_to_json(data, filename: Path):

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"Data saved to {filename}")

def process_all_agreement_pdfs():

    """
    Loop over all main agreement PDFs (skip Q&A PDFs that contain '_QA' in the filename)
    and produce a JSON file per agreement.
    """

    for pdf_file in AGREEMENTS_DIR.glob("Agreement_*.pdf"):

        if "_QA" in pdf_file.stem:
            continue  # Skip Q&A PDFs
        print(f"Processing {pdf_file.name} ...")

        # Convert PDF to markdown text.
        md_text = pdf_to_markdown(pdf_file)

        # Save the markdown to the output directory using the correct path.
        output_md_path = AGREEMENTS_DIR / f"{pdf_file.stem}.md"
        output_md_path.write_text(md_text, encoding="utf-8")
        print(f"Saved markdown to {output_md_path}")

        sections = preprocess_markdown_file(pdf_file.with_suffix(".md"))
        # In case you haven't already saved markdown to disk, you can also do:
        # sections = split_markdown_by_headers(clean_text(md_text))
        # Save processed sections to JSON:
        output_json = PROCESSED_DIR / f"{pdf_file.stem}.json"
        save_to_json(sections, output_json)


##############################
# FURTHER CHUNKING USING TOKEN COUNTS
##############################

def chunk_section_by_tokens(section: Dict[str, str], max_tokens: int = MAX_CHUNK_TOKENS) -> List[Dict[str, str]]:

    """
    Use the Hugging Face tokenizer to count tokens and split a section's content into sub‐chunks.
    The method splits on sentence boundaries if possible.
    """

    text = section["content"]

    # Tokenize using the model's tokenizer (which returns token IDs)
    tokens = tokenizer.tokenize(text)

    if len(tokens) <= max_tokens:
        return [section]

    # For a better split, we can try to split by sentences.
    # Here we use a naive regex sentence split; you might also use nltk.sent_tokenize.
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current_chunk = ""
    current_tokens = []

    for sent in sentences:
        sent_tokens = tokenizer.tokenize(sent)

        # If adding the sentence exceeds max_tokens, store the current chunk.
        if len(current_tokens) + len(sent_tokens) > max_tokens:
            if current_chunk:
                chunks.append({
                    "title": section["title"],
                    "content": current_chunk.strip()
                })
            # Start a new chunk with this sentence.
            current_chunk = sent + " "
            current_tokens = sent_tokens
        else:
            current_chunk += sent + " "
            current_tokens += sent_tokens

    if current_chunk:
        chunks.append({
            "title": section["title"],
            "content": current_chunk.strip()
        })

    return chunks

def further_chunk_sections(sections: List[Dict[str, str]], max_tokens: int = MAX_CHUNK_TOKENS) -> List[Dict[str, str]]:

    """
    Apply token-based chunking to all sections.
    """

    final_chunks = []

    for sec in sections:
        sub_chunks = chunk_section_by_tokens(sec, max_tokens=max_tokens)
        final_chunks.extend(sub_chunks)

    return final_chunks


# GOLD STANDARD Q&A EXTRACTION

In [9]:
def save_to_json_QA(data, filename):

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Gold standard JSON saved to {filename}")



def extract_text_from_pdf(pdf_path):

    """Extract text from a PDF file using pdfplumber."""

    text = ""

    with pdfplumber.open(pdf_path) as pdf:

        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"

    return text

def extract_qa_pairs(text):

    """
    Extract Q&A pairs from text.
    Assumes a format where each pair starts with 'Question <number>:'
    and then 'Answer :' with the answer continuing until the next question or end-of-text.
    """

    qa_pattern = re.compile(
        r"Question\s*(\d+)\s*:\s*(.*?)\s*Answer\s*:\s*(.*?)(?=Question\s*\d+\s*:|$)",
        re.DOTALL | re.IGNORECASE
    )

    qa_pairs = []

    for match in qa_pattern.finditer(text):

        question_num, question, answer = match.groups()
        qa_pairs.append({
            "question_number": question_num.strip(),
            "question": question.strip(),
            "answer": answer.strip()
        })

    return qa_pairs


def build_gold_standard():

    """
    Loop over all Q&A PDFs and build a dictionary.
    Save as a single JSON file.
    """

    # Built a dictionary to hold the gold standard for all agreements
    gold_standard = {}

    # Looping over all Q&A PDFs in the agreements directory

    for pdf_file in AGREEMENTS_DIR.glob("*_QA.pdf"):

        # Extracting an agreement identifier from the filename, "Agreement_N"
        agreement_id = pdf_file.stem.split("_QA")[0]
        print(f"Processing Q&A for {agreement_id} from {pdf_file.name}")

        # Extract text and then Q&A pairs using pdfplumber
        text = extract_text_from_pdf(pdf_file)
        qa_pairs = extract_qa_pairs(text)

        # Store the result in the gold standard dictionary
        gold_standard[agreement_id] = qa_pairs

    # Order the dictionary by the numeric part of the agreement id.
    # Assuming agreement IDs are in the form "Agreement_<number>"
    ordered_gold_standard = dict(

        sorted(
            gold_standard.items(),
            key=lambda x: int(x[0].split('_')[1]) if x[0].split('_')[1].isdigit() else 0
        )

    )

    save_to_json(ordered_gold_standard, GOLD_STANDARD_JSON)



# GENERTAIVE RETRIEVAL FUNCTIONS (FAISS only and FAISS + BM25 Hybrid.)

In [10]:
class FaissRetriever:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(model_name)
        self.index = None
        self.chunk_texts = None

    def build_index(self, chunks: List[Dict[str, str]]):
        self.chunk_texts = [chunk["content"] for chunk in chunks]
        embeddings = self.embedder.encode(self.chunk_texts, convert_to_numpy=True)
        dim = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dim)
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)

    def search(self, query: str, top_k: int = 3) -> List[Dict[str, any]]:
        q_embed = self.embedder.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(q_embed)
        distances, indices = self.index.search(q_embed, top_k)
        results = []
        for dist, idx in zip(distances[0], indices[0]):
            results.append({"text": self.chunk_texts[idx], "score": float(dist)})
        return results

class BM25Retriever:
    # Simple BM25 retrieval based on tokenized content.
    def index(self, chunks: List[Dict[str, str]]):
        from rank_bm25 import BM25Okapi
        self.docs = [chunk["content"] for chunk in chunks]
        self.tokenized_docs = [doc.split() for doc in self.docs]  # very basic tokenization
        self.bm25 = BM25Okapi(self.tokenized_docs)

    def search(self, query: str, top_k: int = 3) -> List[Dict[str, any]]:
        query_tokens = query.split()
        scores = self.bm25.get_scores(query_tokens)
        ranked_idx = np.argsort(scores)[::-1][:top_k]
        results = [{"text": self.docs[idx], "score": float(scores[idx])} for idx in ranked_idx]
        return results

class HybridRetriever:
    """
    Combine FAISS and BM25 scores by taking a simple average of the normalized scores.
    """
    def __init__(self):
        self.faiss_retriever = FaissRetriever()
        self.bm25_retriever = BM25Retriever()

    def index(self, chunks: List[Dict[str, str]]):
        self.faiss_retriever.build_index(chunks)
        self.bm25_retriever.index(chunks)
        # Save chunks for later (they should be the same for both methods)
        self.chunk_texts = [chunk["content"] for chunk in chunks]

    def search(self, query: str, top_k: int = 3) -> List[Dict[str, any]]:
        # Retrieve using FAISS and BM25.
        faiss_results = self.faiss_retriever.search(query, top_k)
        bm25_results = self.bm25_retriever.search(query, top_k)
        # Normalize scores (we assume scores are between 0 and 1 for FAISS, but BM25 might need normalization)
        # Here, we take a simple average based on rank positions.
        combined = []
        for i in range(top_k):
            # If one method returns fewer than top_k, use 0 for missing scores.
            faiss_score = faiss_results[i]["score"] if i < len(faiss_results) else 0
            bm25_score = bm25_results[i]["score"] if i < len(bm25_results) else 0
            avg_score = (faiss_score + bm25_score) / 2.0
            # For context, choose the one with higher individual score (or you could choose to combine texts)
            best_text = faiss_results[i]["text"] if faiss_score >= bm25_score else bm25_results[i]["text"]
            combined.append({"text": best_text, "score": avg_score})
        return combined


# GENERATIVE MODELS LOADING

In [11]:

def load_gen_models(model_name):


  # Load generative model and tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
  )
  print(f"{model_name} Loaded with QLoRA")

  # Apply QLoRA configuration
  qlora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
  )
  model = get_peft_model(model, qlora_config)

  print(f"QLoRA Configured for {model_name} Fine-Tuning")

  return tokenizer, model


# HELPER FUNCTION TO GENERATE ANSWER (For better modularity)

In [12]:
# Helper: generates an answer using the model

def generate_answer(prompt, tokenizer, model, max_new_tokens=200):


    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_new_tokens)
    # Move inputs to the model's device
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # You can print the device of one of the inputs:
    print("Input device:", inputs[list(inputs.keys())[0]].device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True)
    print("Generated output tensor device:", outputs.device)  # This should also be on cuda:0

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # If the prompt includes "Final Answer:", extract what follows.
    if "Final Answer:" in generated_text:
        answer = generated_text.split("Final Answer:")[-1].strip()
    else:
        answer = generated_text.strip()
    return answer




def compute_semantic_similarity(pred, gold):


    # Initialize the embedder for semantic similarity.
    embedder = SentenceTransformer("all-MiniLM-L6-v2")

    pred_emb = embedder.encode([pred])
    gold_emb = embedder.encode([gold])
    return float(cosine_similarity(pred_emb, gold_emb)[0][0])


# RUNNING THE RETRIEVAL & QA EXPERIMENT & EVALUATION METRICS

In [13]:
def run_gen_experiments(device, model_name, gold_data, processed_docs, retriever_dict, top_k=3):

  # Load the generative model and tokenizer
  tokenizer, model = load_gen_models(model_name)


  rouge = evaluate.load("rouge")

  results = []

  for agreement_id, qa_pairs in tqdm(gold_data.items(), desc=f"Processing Agreements for {model_name}"):



        #check if their is a jso file for it- if the pdf preproccer was able to read it
        if agreement_id not in processed_docs:
            print(f"Warning: No processed chunks for {agreement_id}")
            continue

        chunks = processed_docs[agreement_id]

        #skip if the json is empty (agreement - 12)
        if not chunks or all(not re.search(r'\w', chunk.get("content", "")) for chunk in chunks):
            print(f"Skipping {agreement_id} due to empty content.")
            continue


        chunks = further_chunk_sections(chunks, max_tokens=400)


        for retrieval_strategy, retriever in retriever_dict.items():

            #  For Faiss, use build_index(); for hybrid we use index()


            if retrieval_strategy == "faiss":
                start_time = time.time()
                retriever.build_index(chunks)

            else:
                start_time = time.time()
                retriever.index(chunks)


            for qa in tqdm(qa_pairs, desc=f"QA pairs for {agreement_id} ({retrieval_strategy})", leave=False):

              question = qa["question"]
              gold_answer = qa["answer"]
              retrieved = retriever.search(question, top_k=top_k)
              context = " ".join([item["text"] for item in retrieved])
              prompt = (
                    "Answer the question in one or two sentences.\n"
                    "Do not repeat the question or the context; only provide the final answer.\n"
                    f"Question: {question}\n"
                    f"Context: {context}\n"
                    "Final Answer:"
                )


              inputs = tokenizer(prompt, return_tensors="pt", max_length=2560, truncation=True)
              inputs = {k: v.to(device) for k, v in inputs.items()}


              try:
                  generated_ids = model.generate(**inputs, max_length=2560)
              except Exception as e:
                  print(f"Generation error: {e}. Skipping this example.")
                  continue


              duration = time.time() - start_time

              generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

              # Extract only the final answer part
              if "Final Answer:" in generated_text:
                pred_answer = generated_text.split("Final Answer:")[-1].strip()

              else:
                pred_answer = generated_text.strip()


              print("Checkpoint - Calculating metrics")


              # Compute metrics for the single example
              # ROUGE expects lists; we take the "rougeL" score
              rouge_scores = rouge.compute(predictions=[pred_answer], references=[gold_answer])
              rouge_l = rouge_scores.get("rougeL", 0)

              # Compute BERTScore (using bert_score package)
              from bert_score import score as bertscore_score
              P, R, F = bertscore_score([pred_answer], [gold_answer], lang="en")
              bertscore = float(torch.mean(F))

              # Compute semantic similarity (using your provided function)
              semantic_sim = compute_semantic_similarity(pred_answer, gold_answer)

              results.append({

                    "retrieval_strategy": retrieval_strategy,
                    "Model": model_name,
                    "agreement_id": agreement_id,
                    "question": question,
                    "gold_answer": gold_answer,
                    "pred_answer": pred_answer,
                    "time": duration,
                    "ROUGE-L": rouge_l,
                    "BERTScore": bertscore,
                    "Semantic Similarity": semantic_sim
                })


  return results


In [14]:
def run_all_gen_experiments(device, gold_data, processed_docs, retriever_dict, top_k=3):

    all_results = []
    model_names = ["simmo/legal-llama-3"]

    for model_name in model_names:

        print(f"\n--- Running experiment for {model_name} ---")

        results = run_gen_experiments(device, model_name, gold_data, processed_docs, retriever_dict, top_k)
        all_results.extend(results)
        torch.cuda.empty_cache()

    return all_results


In [15]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



def plot_metrics_by_group(results):

    """
    Group results by retrieval strategy and model, then plot two charts:
      - One for Semantic Similarity with one line per group.
      - One for BERTScore with one line per group.
    Uses the metrics already stored in each result.
    """

    from collections import defaultdict
    import matplotlib.pyplot as plt
    import numpy as np

    # Group results by (retrieval_strategy, model).
    groups = defaultdict(list)
    for res in results:
        key = (res["retrieval_strategy"], res["Model"])
        groups[key].append(res)

    # Plot for Semantic Similarity.
    plt.figure(figsize=(12, 8))
    for key, group_results in groups.items():
        # Extract the pre-computed semantic similarity scores.
        semantic_scores = [res["Semantic Similarity"] for res in group_results]
        indices = np.arange(len(semantic_scores))
        retrieval_strategy, model = key
        label = f"{retrieval_strategy} - {model}"
        plt.plot(indices, semantic_scores, label=label, marker="o")
    plt.xlabel("Example Index")
    plt.ylabel("Semantic Similarity")
    plt.title("Semantic Similarity Distribution per Example")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(IMAGES_DIR / "chart1.png", dpi=300)
    plt.show()

    # Plot for BERTScore.
    plt.figure(figsize=(12, 6))
    for key, group_results in groups.items():
        # Extract the pre-computed BERTScore values.
        bert_scores = [res["BERTScore"] for res in group_results]
        indices = np.arange(len(bert_scores))
        retrieval_strategy, model = key
        label = f"{retrieval_strategy} - {model}"
        plt.plot(indices, bert_scores, label=label, marker="o")
    plt.xlabel("Example Index")
    plt.ylabel("BERTScore")
    plt.title("BERTScore Distribution per Example")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(IMAGES_DIR / "chart2.png", dpi=300)
    plt.show()



def plot_time_by_group(results):

    """
    Group results by retrieval strategy and model, then plot the per-example time distribution.
    This produces a chart with one line per group.
    """

    from collections import defaultdict
    import matplotlib.pyplot as plt
    import numpy as np

    # Group results by (retrieval_strategy, model).
    groups = defaultdict(list)
    for res in results:
        key = (res["retrieval_strategy"], res["Model"])
        groups[key].append(res)

    # Prepare the plot for Time Distribution.
    plt.figure(figsize=(12, 8))
    for key, group_results in groups.items():
        # Extract time values from each result.
        times = [res.get("time", 0) for res in group_results]
        indices = np.arange(len(times))
        retrieval_strategy, model = key
        label = f"{retrieval_strategy} - {model}"
        plt.plot(indices, times, label=label, marker="o")

    plt.xlabel("Example Index")
    plt.ylabel("Time (seconds)")
    plt.title("Time Distribution per Example")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(IMAGES_DIR / "chart_time.png", dpi=300)
    plt.show()



def aggregate_results(results):
    """
    Aggregate per-example metrics by grouping results according to
    (retrieval_strategy, Model) and computing the average of each metric.
    Returns a dictionary with aggregated metrics per group.
    """
    from collections import defaultdict
    import numpy as np

    groups = defaultdict(list)
    for res in results:
        key = (res["retrieval_strategy"], res["Model"])
        groups[key].append(res)

    aggregated = {}
    for key, group in groups.items():
        semantic_sim_mean = np.mean([r["Semantic Similarity"] for r in group])
        bertscore_mean = np.mean([r["BERTScore"] for r in group])
        rouge_l_mean = np.mean([r["ROUGE-L"] for r in group])
        time_mean = np.mean([r["time"] for r in group])
        aggregated[key] = {
            "Semantic Similarity": semantic_sim_mean,
            "BERTScore": bertscore_mean,
            "ROUGE-L": rouge_l_mean,
            "Avg Time": time_mean
        }
    return aggregated




def plot_aggregated_results(aggregated_metrics):
    """
    Plot horizontal bar charts for each aggregated metric.
    Each chart will show the metric value per group.
    """
    # List of metric names to plot
    metrics = ["Semantic Similarity", "BERTScore", "ROUGE-L", "Avg Time"]

    # Get groups and corresponding labels
    groups = list(aggregated_metrics.keys())
    labels = [f"{grp[0]} - {grp[1]}" for grp in groups]

    # For each metric, extract its values from the aggregated results
    for metric in metrics:
        values = [aggregated_metrics[grp][metric] for grp in groups]

        plt.figure(figsize=(10, 6))
        # Create a horizontal bar chart.
        y_pos = np.arange(len(labels))
        plt.barh(y_pos, values, color='skyblue')
        plt.yticks(y_pos, labels)
        plt.xlabel(metric)
        plt.title(f"{metric} by Group")
        plt.tight_layout()
        plt.show()


def main():


    # 1. Preprocess and build gold standard as before. ALREADY BUILT
    # process_all_agreement_pdfs()


    build_gold_standard()

    with open(GOLD_STANDARD_JSON, "r", encoding="utf-8") as f:
        gold_data = json.load(f)


    print("Gold data keys:", list(gold_data.keys()))

    processed_docs = {}


    for json_file in PROCESSED_DIR.glob("Agreement_*.json"):
        agreement_id = json_file.stem
        with open(json_file, "r", encoding="utf-8") as f:
            processed_docs[agreement_id] = json.load(f)



    # 2. Initialize retrieval strategies:

    # Strategy 1: FAISS only.
    faiss_only = FaissRetriever()

    # Strategy 2: Hybrid: FAISS + BM25.
    hybrid = HybridRetriever()

    retrieval_strategies = {
        "faiss": faiss_only,
        "hybrid": hybrid
    }

    #3. Run experiments using the generative models.

    results = run_all_gen_experiments(device, gold_data, processed_docs, retrieval_strategies, top_k=3)

    print("Generated results keys:", results[0].keys())

    # Save detailed results.
    df_detailed = pd.DataFrame(results)
    detailed_csv_path = BASE_DIR / "detailed_results_gen_legal_llamma.csv"
    df_detailed.to_csv(detailed_csv_path, index=False)
    print(f"Detailed generative results saved to {detailed_csv_path}")


    plot_metrics_by_group(results)

    plot_time_by_group(results)


    aggregated_metrics = aggregate_results(results)
    print(aggregated_metrics)
    plot_aggregated_results(aggregated_metrics)

In [ ]:
if __name__ == "__main__":
    main()

Processing Q&A for Agreement_16 from Agreement_16_QA.pdf
Processing Q&A for Agreement_8 from Agreement_8_QA.pdf
Processing Q&A for Agreement_1 from Agreement_1_QA.pdf
Processing Q&A for Agreement_4 from Agreement_4_QA.pdf
Processing Q&A for Agreement_3 from Agreement_3_QA.pdf
Processing Q&A for Agreement_5 from Agreement_5_QA.pdf
Processing Q&A for Agreement_11 from Agreement_11_QA.pdf
Processing Q&A for Agreement_13 from Agreement_13_QA.pdf
Processing Q&A for Agreement_18 from Agreement_18_QA.pdf
Processing Q&A for Agreement_6 from Agreement_6_QA.pdf
Processing Q&A for Agreement_2 from Agreement_2_QA.pdf
Processing Q&A for Agreement_7 from Agreement_7_QA.pdf
Processing Q&A for Agreement_12 from Agreement_12_QA.pdf
Processing Q&A for Agreement_14 from Agreement_14_QA.pdf
Processing Q&A for Agreement_9 from Agreement_9_QA.pdf
Processing Q&A for Agreement_15 from Agreement_15_QA.pdf
Processing Q&A for Agreement_17 from Agreement_17_QA.pdf
Processing Q&A for Agreement_10 from Agreement_10

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

QA pairs for Agreement_15 (faiss):  63%|██████▎   | 19/30 [16:40<05:46, 31.53s/it]

Checkpoint - Calculating metrics


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

QA pairs for Agreement_15 (faiss):  67%|██████▋   | 20/30 [16:54<04:23, 26.37s/it]

Checkpoint - Calculating metrics


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

QA pairs for Agreement_15 (faiss):  70%|███████   | 21/30 [20:29<12:26, 82.90s/it]